In [13]:
# Imports
import numpy as np
import pandas as pd 
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, roc_auc_score
)
from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
df = pd.read_csv("ai_human_content_detection_dataset.csv")

In [ ]:
# ============================================================
# 0. Imports
# ============================================================
import os
import numpy as np
import pandas as pd

# Classical ML
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from scipy.stats import randint

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Transformers
import torch
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# ============================================================
# 1. Load Data
# ============================================================
df = pd.read_csv("ai_human_content_detection_dataset.csv")

print("Shape:", df.shape)
print(df.head())

# ============================================================
# 2. Column Definitions
# ============================================================
# Target
y = df["label"]

# Base numeric & categorical columns from the dataset
numeric = [
    "word_count", "avg_word_length", "avg_sentence_length",
    "flesch_reading_ease", "gunning_fog_index",
    "grammar_errors", "punctuation_count"
]

categorical = ["content_type"]

# Filter to ensure columns actually exist
numeric = [c for c in numeric if c in df.columns]
categorical = [c for c in categorical if c in df.columns]

# We'll create a master feature frame that includes both text + classical features
feature_cols = ["text_content"] + numeric + categorical
feature_cols = [c for c in feature_cols if c in df.columns]

X = df[feature_cols].copy()

print("\nUsing feature columns:")
print(feature_cols)

# ============================================================
# 3. Classical Preprocessing (for RF)
# ============================================================
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric),
        ("cat", categorical_transformer, categorical),
    ],
    remainder="drop"  # ignore text_content here
)

# ============================================================
# 4. Train/Test Split
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("\nTrain shape:", X_train.shape, " Test shape:", X_test.shape)

# ============================================================
# 5. Baseline: Random Forest + RSCV (Classical Features Only)
# ============================================================
rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)

rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", rf_base)
])

param_dist = {
    "classifier__n_estimators": randint(200, 1000),
    "classifier__max_depth": [None] + list(range(5, 40, 5)),
    "classifier__min_samples_split": randint(2, 20),
    "classifier__min_samples_leaf": randint(1, 10),
    "classifier__max_features": ["sqrt", "log2", 0.8, 0.5],
    "classifier__bootstrap": [True, False],
    "classifier__class_weight": [None, "balanced", "balanced_subsample"],
}

print("\n=== RandomizedSearchCV: Random Forest (Baseline) ===")
rf_search = RandomizedSearchCV(
    estimator=rf_pipeline,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring="accuracy",
    verbose=1,
    n_jobs=1,
    random_state=42,
    return_train_score=True
)

rf_search.fit(X_train, y_train)

# RF CV results
cv_results = pd.DataFrame(rf_search.cv_results_)
best_idx = rf_search.best_index_
best_train_acc = cv_results.loc[best_idx, "mean_train_score"]
best_val_acc = cv_results.loc[best_idx, "mean_test_score"]

print("\nBest RF Parameters:")
print(rf_search.best_params_)
print(f"RF CV Mean Train Accuracy: {best_train_acc:.3f}")
print(f"RF CV Mean Validation Accuracy: {best_val_acc:.3f}")

# Evaluate best RF on held-out test set
best_rf_model = rf_search.best_estimator_
y_pred_train_rf = best_rf_model.predict(X_train)
y_pred_test_rf = best_rf_model.predict(X_test)
y_prob_test_rf = best_rf_model.predict_proba(X_test)[:, 1]

train_acc_rf = accuracy_score(y_train, y_pred_train_rf)
test_acc_rf = accuracy_score(y_test, y_pred_test_rf)
roc_rf = roc_auc_score(y_test, y_prob_test_rf)

print("\n=== Baseline RF Performance ===")
print(f"Train Accuracy: {train_acc_rf:.3f}")
print(f"Val Accuracy (CV best): {best_val_acc:.3f}")
print(f"Test Accuracy: {test_acc_rf:.3f}")
print(f"ROC-AUC: {roc_rf:.3f}")

print("\nRF Classification Report:")
print(classification_report(y_test, y_pred_test_rf))

# Confusion matrix for RF
conf_mat_rf = confusion_matrix(y_test, y_pred_test_rf)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_mat_rf, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Human", "AI"], yticklabels=["Human", "AI"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Random Forest (Baseline)")
plt.tight_layout()
plt.show()

# RF Feature importances (top 15)
rf_clf = best_rf_model.named_steps["classifier"]

# Get feature names after preprocessing
feature_names_num = numeric
feature_names_cat = list(
    best_rf_model.named_steps["preprocessor"]
    .named_transformers_["cat"]
    .named_steps["encoder"]
    .get_feature_names_out(categorical)
)
all_features = feature_names_num + feature_names_cat

importances = pd.Series(rf_clf.feature_importances_, index=all_features)
importances = importances.sort_values(ascending=False).head(15)

plt.figure(figsize=(8, 6))
sns.barplot(x=importances, y=importances.index)
plt.title("Top 15 Feature Importances - RF (Baseline)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# ============================================================
# 6b. DeBERTa v3 Base Fine-Tuning (Pure Classifier)
# ============================================================

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from torch.utils.data import Dataset
import numpy as np

DEBERTA_MODEL = "microsoft/deberta-v3-base"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("\nUsing device for DeBERTa:", device)

# ------------------------------------------------------------
# (A) HuggingFace Dataset Wrapper
# ------------------------------------------------------------

class HFTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = list(texts)
        self.labels = labels.tolist() if labels is not None else None
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {k: v.squeeze(0) for k, v in encoding.items()}

        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item

# ------------------------------------------------------------
# (B) Train/Val Split for DeBERTa (from TRAIN only)
# ------------------------------------------------------------

from sklearn.model_selection import train_test_split

X_train_text = X_train["text_content"].astype(str)
X_test_text  = X_test["text_content"].astype(str)

X_train_deb, X_val_deb, y_train_deb, y_val_deb = train_test_split(
    X_train_text,
    y_train,
    test_size=0.2,
    stratify=y_train,
    random_state=42
)

print("\nDeBERTa text sizes:")
print("  Train:", len(X_train_deb))
print("  Val:  ", len(X_val_deb))
print("  Test: ", len(X_test_text))

tokenizer = AutoTokenizer.from_pretrained(DEBERTA_MODEL)

train_dataset = HFTextDataset(X_train_deb, y_train_deb, tokenizer, max_length=512)
val_dataset   = HFTextDataset(X_val_deb,   y_val_deb,   tokenizer, max_length=512)
test_dataset  = HFTextDataset(X_test_text, y_test,      tokenizer, max_length=512)

# ------------------------------------------------------------
# (C) Model + TrainingArguments + Trainer
# ------------------------------------------------------------

model = AutoModelForSequenceClassification.from_pretrained(
    DEBERTA_MODEL,
    num_labels=2
).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_score(labels, preds)
    f1  = f1_score(labels, preds)

    # For ROC-AUC, need probabilities of class 1
    probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float("nan")

    return {"accuracy": acc, "f1": f1, "roc_auc": auc}

training_args = TrainingArguments(
    output_dir="./deberta_finetuned",
    eval_strategy="epoch",       # <-- CORRECT ARG NAME
    save_strategy="epoch",
    logging_dir="./logs_deberta",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n=== Fine-tuning DeBERTa v3 Base ===")
train_result = trainer.train()

print("\n=== DeBERTa Validation Metrics (Best Checkpoint) ===")
eval_metrics = trainer.evaluate()
for k, v in eval_metrics.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}")
    else:
        print(f"{k}: {v}")

# Save fine-tuned model
model.save_pretrained("./deberta_finetuned")
tokenizer.save_pretrained("./deberta_finetuned")

# ------------------------------------------------------------
# (D) Test-Set Evaluation for DeBERTa
# ------------------------------------------------------------

print("\n=== Evaluating DeBERTa on Test Set ===")
test_predictions = trainer.predict(test_dataset)

test_logits = test_predictions.predictions
test_labels = test_predictions.label_ids

test_probs = torch.softmax(torch.tensor(test_logits), dim=1)[:, 1].numpy()
test_preds = (test_probs >= 0.5).astype(int)

deb_acc  = accuracy_score(test_labels, test_preds)
deb_roc  = roc_auc_score(test_labels, test_probs)
deb_f1   = f1_score(test_labels, test_preds)

print("\n=== DeBERTa Test Performance ===")
print(f"Accuracy: {deb_acc:.3f}")
print(f"ROC-AUC:  {deb_roc:.3f}")
print(f"F1-score: {deb_f1:.3f}")

print("\nDeBERTa Classification Report:")
print(classification_report(test_labels, test_preds))

# Confusion matrix for DeBERTa
conf_mat_deb = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_mat_deb, annot=True, fmt="d", cmap="Purples",
            xticklabels=["Human", "AI"], yticklabels=["Human", "AI"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - DeBERTa v3 Base")
plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# (E) Simple RF + DeBERTa Probability Ensemble (Optional)
# ------------------------------------------------------------

# Assumes you already computed RF test probabilities earlier as:
#   y_prob_test_rf = best_rf_model.predict_proba(X_test)[:, 1]

print("\n=== RF + DeBERTa Simple Probability Ensemble ===")

# Simple average ensemble
ensemble_probs = 0.5 * y_prob_test_rf + 0.5 * test_probs
ensemble_preds = (ensemble_probs >= 0.5).astype(int)

ens_acc = accuracy_score(y_test, ensemble_preds)
ens_roc = roc_auc_score(y_test, ensemble_probs)
ens_f1  = f1_score(y_test, ensemble_preds)

print(f"Ensemble Accuracy: {ens_acc:.3f}")
print(f"Ensemble ROC-AUC:  {ens_roc:.3f}")
print(f"Ensemble F1-score: {ens_f1:.3f}")

print("\nEnsemble Classification Report:")
print(classification_report(y_test, ensemble_preds))

conf_mat_ens = confusion_matrix(y_test, ensemble_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_mat_ens, annot=True, fmt="d", cmap="Oranges",
            xticklabels=["Human", "AI"], yticklabels=["Human", "AI"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - RF + DeBERTa Ensemble")
plt.tight_layout()
plt.show()


Shape: (1367, 17)
                                        text_content      content_type  \
0  Score each cause. Quality throughout beautiful...    academic_paper   
1  Board its rock. Job worker break tonight coupl...             essay   
2  Way debate decision produce. Dream necessary c...    academic_paper   
3  Story turn because such during open model. Tha...  creative_writing   
4  Place specific as simply leader fall analysis....      news_article   

   word_count  character_count  sentence_count  lexical_diversity  \
0         288             1927              54             0.9514   
1         253             1719              45             0.9723   
2         420             2849              75             0.9071   
3         196             1310              34             0.9592   
4         160             1115              28             0.9688   

   avg_sentence_length  avg_word_length  punctuation_ratio  \
0                 5.33             5.69             0.0280  